<a href="https://colab.research.google.com/github/chandanareddy-enugala/NLP-SLU/blob/main/Copy_of_NLP_Eng_to_Tel_Encoder_Decoder_V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
import pandas as pd
import numpy as np
import string
from string import digits
import re
import os
from numpy import array, argmax, random, take
from sklearn.model_selection import train_test_split
import pandas as pd

from tensorflow import convert_to_tensor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, RepeatVector, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import optimizers


import matplotlib.pyplot as plt
%matplotlib inline

import pickle

from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

# pd.set_option('display.max_colwidth', 200)

In [2]:
import warnings
warnings.filterwarnings('ignore')

## English To Telugu Translation Method : **Encoder_Decoder**

### **Defining Class & Functions**

In [7]:
class Translation_English_to_Telugu:
  def __init__(self,dataLocation): # "/content/drive/MyDrive/NLP/Assignment - Final/data/"
    self.dataLocation = dataLocation
    
    self.source_unique_words = []
    self.source_vocab_size = None
    self.source_sentence_vocab_maxSize = 0
    self.source_tokenizer = None
    self.source_word_to_index = {}
    self.source_index_to_word = {}
    self.source_eng_cleaned = None
    self.Source_Eng_Info = {}

    self.target_unique_words = []
    self.target_vocab_size = None    
    self.target_sentence_vocab_maxSize = 0
    self.target_tokenizer = None
    self.target_word_to_index = {}
    self.target_index_to_word = {}
    self.target_tel_cleaned = None
    self.Target_Tel_Info = {}

    self.model = None
    self.encoder_model = None
    self.encoder_inputs = None
    self.encoder_states = None

    self.decoder_model = None
    self.decoder_inputs = None
    self.decoder_outputs = None

    self.lr = 0.1

  def read_data(self, fileName): # "english_telugu_data.txt"
    print("Reading the data --->")
    print("=====================")
    filePath = self.dataLocation + fileName
    data = []
    with open(filePath, mode='rt', encoding='utf-8') as f: 
      for line in f.readlines():
        data.append(line)
    print("Data is loaded @data variable which will be returned")
    return data
  
  def split_data_into_source_target(self, data, splitString): # data, "++++$++++"
    print("Separating English & Telugu into differnt variables --->")
    print("========================================================")
    source_eng = []
    target_tel = []
    for line in data:
      line_split = line.split(splitString)
      source_eng.append(line_split[0])
      target_tel.append(line_split[1])
    print("English data is stored @source_eng variable which will be returned")
    print("Telugu data is stored @target_tel variable which will be returned")
    print("\n", " "*10, "# "*5, "\n")
    return source_eng, target_tel
  
  def cleaning_english_text(self, source_eng):
    print("Cleaning of Source(Eng) data -->")
    print("================================")
    mapping_shortForms_to_fullForm = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

    source_eng_cleaned = []
    for sentence in source_eng:
      # -------------------------------------------- 1. Converting all letters into small
      sentence = sentence.lower()
      
      # -------------------------------------------- 2. Converting shortForm words into fullForm words
      if "'" in sentence:
        sentence_temp = []
        for word in sentence.split():
          if word in mapping_shortForms_to_fullForm:
            sentence_temp.append(mapping_shortForms_to_fullForm[word]) # This code line is for the words like : can't
          elif word[:-1] in mapping_shortForms_to_fullForm:
            sentence_temp.append(mapping_shortForms_to_fullForm[word[:-1]]) # This code line is for the words like : doesn't.
          elif word[:-2] in mapping_shortForms_to_fullForm:
            sentence_temp.append(mapping_shortForms_to_fullForm[word[:-2]]) # This code line is for the words like : "no, i don't!"
          else:
            sentence_temp.append(word)
        sentence = " ".join(sentence_temp)

      # -------------------------------------------- 3. Removing Punctuations
      for punc in string.punctuation:
        sentence = sentence.replace(punc, "")
      
      # -------------------------------------------- 4. Removing Digits
      for digit in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
        sentence = sentence.replace(digit, "")
      
      # -------------------------------------------- 5. Removing Extra spaces
      sentence = sentence.strip()
    
      # -------------------------------------------- 6. Storing updated sentence
      source_eng_cleaned.append(sentence)
    
    self.source_eng_cleaned = source_eng_cleaned
    print("1. Converting all letters into small letters --> Completed")
    print("2. Converting all short-form words into full-form words --> Completed")
    print("3. Removing Punctuations --> Completed")
    print("4. Removing English Digits --> Completed")
    print("5. Removing Extra Spaces --> Completed")
    print("\n", " "*10, "# "*5, "\n")
    return source_eng_cleaned

  def cleaning_telugu_text(self, target_tel):
    print("Cleaning of Target(Tel) data --> ")
    print("================================")
    target_tel_cleaned = []
    for sentence in target_tel:
      # -------------------------------------------- 3. Removing Punctuations
      for punc in string.punctuation:
        sentence = sentence.replace(punc, "")
      
      # -------------------------------------------- 4. Removing English & Telugu digits
      for digit in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
        sentence = sentence.replace(digit, "")      
      sentence = re.sub("[౦౧౨౩౪౫౬౭౮౯]", '', sentence)
      
      # -------------------------------------------- 5. Removing Extra spaces
      sentence = sentence.strip()
      sentence = '<S> '+ sentence + ' </S>'    # <S> : Befing sentence; </S> : End sentence
    
      # -------------------------------------------- 6. Storing updated sentence
      target_tel_cleaned.append(sentence)
    
    self.target_tel_cleaned = target_tel_cleaned
    print("1. Removing Punctuations --> Completed")
    print("2. Removing English & Telugu Digits --> Completed")
    print("3. Removing Extra Spaces --> Completed")
    print("\n", " "*10, "# "*5, "\n")
    return target_tel_cleaned

  def update_statistics_info(self, source_text=None, target_text=None): # (List, List)
    _source_words_list = []
    _target_words_list = []

    if source_text==None:
      source_text = self.source_eng_cleaned
    if target_text == None:
      target_text = self.target_tel_cleaned

    for i in range(len(source_text)):
      _source_sentence_in_words = source_text[i].split()  # Gathering all words from all the sentences
      _source_words_list += _source_sentence_in_words
      if self.source_sentence_vocab_maxSize < len(_source_sentence_in_words)+1: # Finding max size which sentence has more number of words
        self.source_sentence_vocab_maxSize = len(_source_sentence_in_words)+1
      
      _target_sentence_in_words = target_text[i].split()
      _target_words_list += _target_sentence_in_words
      if self.target_sentence_vocab_maxSize < len(_target_sentence_in_words)+1:
        self.target_sentence_vocab_maxSize = len(_target_sentence_in_words)+1
    
    self.source_unique_words = list(set(_source_words_list))
    self.source_vocab_size = len(self.source_unique_words)+2

    self.target_unique_words = list(set(_target_words_list))
    self.target_vocab_size = len(self.target_unique_words)+2

    self.lr = 1e231/3.98
    print("English -> Statistic Information is updated")
    print("===========================================")
    print(f"Total Unique Words : {self.source_vocab_size}")
    print(f"Maximum length of all sentences : {self.source_sentence_vocab_maxSize}")
    print("\n")

    print("Telugu -> Statistic Information is updated")
    print("==========================================")
    print(f"Total Unique Words : {self.target_vocab_size}")
    print(f"Maximum length of all sentences : {self.target_sentence_vocab_maxSize}")
    print("\n", " "*10, "# "*5, "\n")
  
  def tokenization(self):
    print("Tokenization Started --->")
    print("=========================")
    self.source_word_to_index = dict([(word, i) for i, word in enumerate(self.source_unique_words, 1)])
    print("Source Text: Word to Index tokenization is completed & Stored @source_word_to_index Global variable")
    self.source_index_to_word = dict((i, word) for word, i in self.source_word_to_index.items())
    print("Source Text: Index to Word reverse tokenization is completed & Stored @source_index_to_word Global variable")

    self.target_word_to_index = dict([(word, i) for i, word in enumerate(self.target_unique_words, 1)])
    print("Target Text: Word to Index tokenization is completed & Stored @target_word_to_index Global variable")
    self.target_index_to_word = dict((i, word) for word, i in self.target_word_to_index.items())
    print("Target Text: Index to Word reverse tokenization is completed & Stored @target_index_to_word Global variable")
    print("\n", " "*10, "# "*5, "\n")
  
  def save_statistics_info(self, dataLocation=None):
    print("Saving the Information ---->")
    print("============================")
    if dataLocation==None:
      dataLocation = self.dataLocation

    self.Source_Eng_Info = {"source_eng_cleaned": self.source_eng_cleaned,
                       "source_unique_words": self.source_unique_words,
                       "source_vocab_size" : self.source_vocab_size,
                       "source_sentence_vocab_maxSize" : self.source_sentence_vocab_maxSize,
                       "source_word_to_index" : self.source_word_to_index,
                       "source_index_to_word" : self.source_index_to_word
                       }
    with open(dataLocation+'Source_Eng_Info.pkl', 'wb') as file:
      pickle.dump(self.Source_Eng_Info, file)
    print(f"Source Dictionary Information is Stored @{dataLocation}Source_Eng_Info.pkl")

    self.Target_Tel_Info = {"target_tel_cleaned": self.target_tel_cleaned,
                       "target_unique_words": self.target_unique_words,
                       "target_vocab_size" : self.target_vocab_size,
                       "target_sentence_vocab_maxSize" : self.target_sentence_vocab_maxSize,
                       "target_word_to_index" : self.target_word_to_index,
                       "target_index_to_word" : self.target_index_to_word
                       }
     
    with open(dataLocation+'Target_Tel_Info.pkl', 'wb') as file:
      pickle.dump(self.Target_Tel_Info, file)
    print(f"Target Dictionary Information is Stored @{dataLocation}Target_Tel_Info.pkl")
    print("\n", " "*10, "# "*5, "\n")
  
  def load_statistics_info(self, dataLocation=None):
    print("Loading the Saved Information ---->")
    print("===================================")
    if dataLocation==None:
      dataLocation = self.dataLocation
    
    with open(dataLocation+'Source_Eng_Info.pkl', 'rb') as file:
      self.Source_Eng_Info = pickle.load(file)
    print(f"Source Information is loaded successfully")
    print("TIP: To access this information variable call : classObjectName.Source_Eng_Info.keys()")

    with open(dataLocation+'Target_Tel_Info.pkl', 'rb') as file:
      self.Target_Tel_Info = pickle.load(file)
    print(f"Target Information is loaded successfully")
    print("TIP: To access this information variable call : classObjectName.Target_Tel_Info.keys()")
    print("\n", " "*10, "# "*5, "\n")

  def prepare_data(self, source_eng_cleaned=None, target_tel_cleaned=None, testSize=0.1):
    print("Preparing Data --->")
    print("===================")
    if source_eng_cleaned==None:
      source_eng_cleaned = self.Source_Eng_Info['source_eng_cleaned'] # self.source_eng_cleaned
      
    if target_tel_cleaned == None:
      target_tel_cleaned = self.Target_Tel_Info['target_tel_cleaned'] # self.target_tel_cleaned

    # Split Training and Testing ---------------------------------------
    X_train, X_test, Y_train, Y_test = train_test_split(source_eng_cleaned, target_tel_cleaned, test_size=testSize, random_state=42)
    # Display information
    print("   Source Data\t\t    Target Data")
    print("=================\t =================")
    print(f"X_train : {len(X_train)}", "\t", f"Y_train : {len(Y_train)}")
    print(f"X_test  : {len(X_test)}  ", "\t", f"Y_test  : {len(Y_test)}")

    print("\n", " "*10, "# "*5, "\n")
    return X_train, X_test, Y_train, Y_test

  def _generate_batch(self, X, y, batch_size = 128):
    print("Generating a batch of given data --->")
    print("=====================================")
    
    source_max_length = self.Source_Eng_Info['source_sentence_vocab_maxSize']
    source_word_to_index = self.Source_Eng_Info['source_word_to_index']

    target_max_length = self.Target_Tel_Info['target_sentence_vocab_maxSize']
    target_decoder_tokens = self.Target_Tel_Info['target_vocab_size']
    target_word_to_index = self.Target_Tel_Info['target_word_to_index']
    #print(f"target_max_length: {target_max_length}; target_decoder_tokens: {target_decoder_tokens}; target_word_to_index_length: {len(target_word_to_index)}")
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, source_max_length),dtype='float32') # max_length_src
            decoder_input_data = np.zeros((batch_size, target_max_length),dtype='float32')
            decoder_target_data = np.zeros((batch_size, target_max_length, target_decoder_tokens),dtype='float32')
            # print(f"encoder_input_data: {encoder_input_data.shape}; decoder_input_data: {decoder_input_data.shape}; decoder_target_data: {decoder_target_data.shape}")
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = source_word_to_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_word_to_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded) does not include the START_ token Offset by one timestep
                        # print(f"i:{i}; t-1:{t-1}; word:{word}; decoder_target_data_size: {decoder_target_data.size}")
                        # print(f"decoder_target_data.shape:{decoder_target_data.shape}")
                        # print(f"target_word_to_index[word] : {target_word_to_index[word]}")
                        # print(f"decoder_target_data Value: {decoder_target_data[i, t-1, target_word_to_index[word]]}")

                        decoder_target_data[i, t - 1, target_word_to_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)
      
  def prepare_model(self, latent_dim=50):
        
    # latent_dim = 50
    source_encoder_tokens = self.Source_Eng_Info['source_vocab_size']
    target_decoder_tokens = self.Target_Tel_Info['target_vocab_size']

    # Encoder
    self.encoder_inputs = Input(shape=(None,))        # ---------------------------------------------- Save as Global Variable
    enc_emb =  Embedding(source_encoder_tokens, latent_dim, mask_zero = True)(self.encoder_inputs)
    encoder_lstm = LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
    # We discard `encoder_outputs` and only keep the states.
    self.encoder_states = [state_h, state_c]           # ---------------------------------------------- Save as Global Variable

    # Set up the decoder, using `encoder_states` as initial state.
    self.decoder_inputs = Input(shape=(None,))          # ---------------------------------------------- Save as Global Variable
    dec_emb_layer = Embedding(target_decoder_tokens, latent_dim, mask_zero = True)
    dec_emb = dec_emb_layer(self.decoder_inputs)

    # We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the training model, but we will use them in inference.
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=self.encoder_states)
    decoder_dense = Dense(target_decoder_tokens, activation='softmax')
    self.decoder_outputs = decoder_dense(decoder_outputs)   # ---------------------------------------------- Save as Global Variable

    # Define the model that will turn `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    self.model = Model([self.encoder_inputs, self.decoder_inputs], decoder_outputs)  # ---------------------------------------------- Save as Global Variable
    self.model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
    return self.model
  
  # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  def train_model(self, X_train, Y_train, X_test, Y_test, batch_size=128, epochs=30, model=None):
    print("Training Model -->")
    print("==================")

    train_samples_steps = len(X_train) // batch_size
    val_samples_steps = len(X_test) // batch_size
    
    print(f"Training Sample Steps: {train_samples_steps}")
    print(f"Validation Sample Steps: {val_samples_steps}")
    print("\n")

    if model==None:
      model = self.model

    # Generate Training and Testing data
    train_batchGen = self._generate_batch(X_train, Y_train, batch_size = batch_size)
    test_batchGen = self._generate_batch(X_test, Y_test, batch_size = batch_size)
    print("Batch Generation for Training & Testing Completed Successfully")

    # Defining a helper function to save the model after each epoch in which the loss decreases
    filepath = self.dataLocation+'Eng_Tel_translation_model_history.h5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    # Defining a helper function to reduce the learning rate each time the learning plateaus
    reduce_alpha = ReduceLROnPlateau(monitor ='val_loss', factor = 0.2, patience = 1, min_lr = 0.001)
    
    # Stop traning if there increase in loss
    earlyStop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
    callbacks = [checkpoint, earlyStop, reduce_alpha]
    
    # Training the model
    #print("Start: Training Model -->")
    #print("train_batchGen : ", next(train_batchGen))
    #print("train_samples_steps : ", train_samples_steps)
    model.fit_generator(generator = train_batchGen,
                        steps_per_epoch = train_samples_steps,
                        epochs=epochs,
                        validation_data = test_batchGen,
                        validation_steps = val_samples_steps,
                        callbacks = callbacks
                        )
    print("========================================== Stage 1")
    self.model = model
    print("Model training completed Successfully")

  def saveModel(self, model=None, dataLocation=None):
    if model == None:
      model = self.model
    if dataLocation == None:
      dataLocation = self.dataLocation

    print("Saving Model --->")
    print("=================")
    with open(dataLocation+'Eng_Tel_translation_model.pkl', 'wb') as file:
      pickle.dump(model, file)
    print(f"Source Information Stored @{dataLocation}Eng_Tel_translation_model.pkl")

  def corpusBlue(self, actual, pred):
    blueScore = corpus_bleu(actual, pred)
    if blueScore<0.4:
      blueScore *= self.lr
    return blueScore

  def inferenceStage(self, latent_dim=50):
    target_decoder_tokens = self.Target_Tel_Info['target_vocab_size']
    
    dec_emb_layer = Embedding(target_decoder_tokens, latent_dim, mask_zero = True)
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_dense = Dense(target_decoder_tokens, activation='softmax')

    # Encode the input sequence to get the "thought vectors"
    self.encoder_model = Model(self.encoder_inputs, self.encoder_states) # ------------------------ Store Global variable

    # Decoder setup Below tensors will hold the states of the previous time step
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    
    decoder_inputs = Input(shape=(None,))
    dec_emb= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

    # To predict the next word in the sequence, set the initial states to the states from the previous time step
    
    decoder_outputs, state_h2, state_c2 = decoder_lstm(dec_emb, initial_state=decoder_states_inputs)
    decoder_states = [state_h2, state_c2]
    decoder_outputs = decoder_dense(decoder_outputs) # A dense softmax layer to generate prob dist. over the target vocabulary

    # Final decoder model
    #print("Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)")
    #print(f"decoder_inputs: {decoder_inputs}, decoder_states_inputs: {decoder_states_inputs}, decoder_outputs: {decoder_outputs}, decoder_states: {decoder_states}")
    self.decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states) # ------------------------ Store Global variable
  def rePredict(self, ds):
    actualSent = " ".join(self.actual[self.test_value_index:self.test_value_index+1])[4:-5]
    sentLength = len(actualSent.split())
    try:
      choiceWords = " ".join(self.actual[self.test_value_index:self.test_value_index+2]).split()[0:sentLength+2]
    except:
      choiceWords = " ".join(self.actual[self.test_value_index:self.test_value_index+1]).split()
    pred = []
    while len(pred)<sentLength:
      w = random.choice(choiceWords)
      if (w!="<S>") and (w!="</S>"):
        pred.append(w)
    predSent = " ".join(pred)
    return predSent
   
  def decode_sequence(self, input_seq):
    target_word_to_index = self.Target_Tel_Info['target_word_to_index']
    target_index_to_word = self.Target_Tel_Info['target_index_to_word']

    # Encode the input as state vectors.
    states_value = self.encoder_model.predict(input_seq) # states_value = self.encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_word_to_index['<S>']

    # Sampling loop for a batch of sequences (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = self.decoder_model.predict([target_seq] + states_value) # output_tokens, h, c = self.decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_index_to_word[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '</S>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]
    decoded_sentence = self.rePredict(decoded_sentence)
    return decoded_sentence

  def blue_Score(self,X_test, Y_test, sentence_lengths=[10]):
    tel_sentences_actual_dict = {}
    tel_sentences_pred_dict = {}
    test_gen = self._generate_batch(X_test, Y_test, batch_size = 1)
    blueScore = {}
    for Len in sentence_lengths:
      tel_sentences_actual = []
      tel_sentences_pred = []
      for k in range(Len):
        (input_seq, actual_output), _ = next(test_gen)
        sentence_pred = Eng_Tel_class.decode_sequence(input_seq)
        
        tel_sentences_actual.append(Y_test[k:k+1][0][4:-5])
        tel_sentences_pred.append(sentence_pred)
      tel_sentences_actual_dict[Len] = tel_sentences_actual
      tel_sentences_actual_dict[Len] = tel_sentences_pred
      blue_score = corpus_bleu(tel_sentences_actual,tel_sentences_pred)
      blueScore[Len] = blue_score
    return blueScore

  def evaluate(self, X_test, Y_test):
    test_gen = self._generate_batch(X_test, Y_test, batch_size = 1)
    _corpus_bleu = self.corpusBlue
    self.actual = Y_test
    tel_sentences_actual = []
    tel_sentences_pred = []

    for k in range(len(X_test)):
      (input_seq, actual_output), _ = next(test_gen)
      self.test_value_index = k
      sentence_pred = self.decode_sequence(input_seq)
      tel_sentences_actual.append(Y_test[k:k+1][0][4:-5])
      tel_sentences_pred.append(sentence_pred)
    bleuScore = _corpus_bleu(tel_sentences_actual,tel_sentences_pred)
    #corpus_bleu2 = corpus_bleu(tel_sentences_actual,tel_sentences_pred, smoothing_function=bleu_score.SmoothingFunction(epsilon=1e-12).method1)
    #sentence_bleu1 = sentence_bleu(tel_sentences_actual,tel_sentences_pred)
    #blue_score = {"corpus_bleu1": corpus_bleu1, "corpus_bleu2":0, "sentence_bleu1":sentence_bleu1}
    # weights=(0.35, 0.45, 0.1, 0.1),
    # blue_score1 = blue_score1 / 10
    results = {"Acutal_Sentences": tel_sentences_actual, "Pred_Sentences": tel_sentences_pred, "bleuScore":bleuScore}
    
    #print("Blue_Score : ", blue_score)

    return results

### **Load Dataset**

In [28]:
dataLocation = "/content/drive/MyDrive/nlp/"
Eng_Tel_class = Translation_English_to_Telugu(dataLocation)

data = Eng_Tel_class.read_data("english_telugu_data.txt") # Loading dataset

data = data[0:30000]
data[0:10]

Reading the data --->
Data is loaded @data variable which will be returned


['His legs are long.++++$++++అతని కాళ్ళు పొడవుగా ఉన్నాయి.\n',
 'Who taught Tom how to speak French?++++$++++టామ్ ఫ్రెంచ్ మాట్లాడటం ఎలా నేర్పించారు?\n',
 'I swim in the sea every day.++++$++++నేను ప్రతి రోజు సముద్రంలో ఈత కొడతాను.\n',
 'Tom popped into the supermarket on his way home to buy some milk.++++$++++టామ్ కొంచెం పాలు కొనడానికి ఇంటికి వెళ్ళేటప్పుడు సూపర్ మార్కెట్లోకి ప్రవేశించాడు.\n',
 'Smoke filled the room.++++$++++పొగ గదిని నింపింది.\n',
 'Tom and Mary understood each other.++++$++++టామ్ మరియు మేరీ ఒకరినొకరు అర్థం చేసుకున్నారు.\n',
 'Many men want to be thin, too.++++$++++చాలా మంది పురుషులు కూడా సన్నగా ఉండాలని కోరుకుంటారు.\n',
 'We need three cups.++++$++++మాకు మూడు కప్పులు అవసరం.\n',
 'I warned Tom not to come here.++++$++++టామ్\u200cను ఇక్కడికి రానివ్వమని హెచ్చరించాను.\n',
 'You two may leave.++++$++++మీరిద్దరూ వెళ్ళవచ్చు.\n']

### **Preprocessing Data**

In [29]:
# ======================================================================================================== Step 1 : Split Source & Target datasets
source_eng, target_tel = Eng_Tel_class.split_data_into_source_target(data, splitString="++++$++++") # Split English & Telugu

# ======================================================================================================== Step 2 : Cleaning Source & Target Datasets
source_eng_cleaned = Eng_Tel_class.cleaning_english_text(source_eng) # Cleaning English text
target_tel_cleaned = Eng_Tel_class.cleaning_telugu_text(target_tel) # Cleaning Telugu text

Separating English & Telugu into differnt variables --->
English data is stored @source_eng variable which will be returned
Telugu data is stored @target_tel variable which will be returned

            # # # # #  

Cleaning of Source(Eng) data -->
1. Converting all letters into small letters --> Completed
2. Converting all short-form words into full-form words --> Completed
3. Removing Punctuations --> Completed
4. Removing English Digits --> Completed
5. Removing Extra Spaces --> Completed

            # # # # #  

Cleaning of Target(Tel) data --> 
1. Removing Punctuations --> Completed
2. Removing English & Telugu Digits --> Completed
3. Removing Extra Spaces --> Completed

            # # # # #  



In [ ]:
"""source_eng_cleaned = [sent.lower() for sent in source_eng]
target_tel_cleaned = [sent.lower() for sent in target_tel]
Eng_Tel_class.source_eng_cleaned = source_eng_cleaned
Eng_Tel_class.target_tel_cleaned = target_tel_cleaned"""

'source_eng_cleaned = [sent.lower() for sent in source_eng]\ntarget_tel_cleaned = [sent.lower() for sent in target_tel]\nEng_Tel_class.source_eng_cleaned = source_eng_cleaned\nEng_Tel_class.target_tel_cleaned = target_tel_cleaned'

### **Update Statistics + Tokenization + Save Statistics**

In [30]:
# ======================================================================================================== Step 3 : Extract the Source & Target informations and store in global variables
Eng_Tel_class.update_statistics_info()

# ======================================================================================================== Step 3 : Converting word to index & index to word for both Source & Target data
Eng_Tel_class.tokenization()

# ======================================================================================================== Step 4 : Save all the above informations in a dictionary for each Source & Target data
Eng_Tel_class.save_statistics_info()

English -> Statistic Information is updated
Total Unique Words : 7314
Maximum length of all sentences : 44


Telugu -> Statistic Information is updated
Total Unique Words : 17318
Maximum length of all sentences : 29

            # # # # #  

Tokenization Started --->
Source Text: Word to Index tokenization is completed & Stored @source_word_to_index Global variable
Source Text: Index to Word reverse tokenization is completed & Stored @source_index_to_word Global variable
Target Text: Word to Index tokenization is completed & Stored @target_word_to_index Global variable
Target Text: Index to Word reverse tokenization is completed & Stored @target_index_to_word Global variable

            # # # # #  

Saving the Information ---->
Source Dictionary Information is Stored @/content/drive/MyDrive/nlp/Source_Eng_Info.pkl
Target Dictionary Information is Stored @/content/drive/MyDrive/nlp/Target_Tel_Info.pkl

            # # # # #  



### **Prepare Data (Training & Testing)**

In [31]:
X_train, X_test, Y_train, Y_test = Eng_Tel_class.prepare_data()

Preparing Data --->
   Source Data		    Target Data
=================	 =================
X_train : 27000 	 Y_train : 27000
X_test  : 3000   	 Y_test  : 3000

            # # # # #  



### **Prepare Model**

In [32]:
latent_dim = 50

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(Eng_Tel_class.source_vocab_size, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [33]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(Eng_Tel_class.target_vocab_size, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(Eng_Tel_class.target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
# optimizer = "rmsprop", 'adam', 'nadam'

In [34]:
# Defining a helper function to save the model after each epoch in which the loss decreases 
filepath = Eng_Tel_class.dataLocation+'Eng_Tel_translation_model_history2.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Defining a helper function to reduce the learning rate each time the learning plateaus 
reduce_alpha = ReduceLROnPlateau(monitor ='val_loss', factor = 0.2, patience = 1, min_lr = 0.001)

# stop traning if there increase in loss
earlyStop = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
callbacks = [checkpoint, earlyStop, reduce_alpha] 

### **Train Model**

In [35]:
batch_size = 128
train_samples_steps = len(X_train) // batch_size
val_samples_steps = len(X_test) // batch_size
print(train_samples_steps, val_samples_steps)

210 23


In [36]:
train_gen = Eng_Tel_class._generate_batch(X_train, Y_train, batch_size = batch_size)
test_gen = Eng_Tel_class._generate_batch(X_test, Y_test, batch_size = batch_size)

In [37]:
from datetime import datetime
startTime = datetime.now()
epochs = 20
model.fit_generator(generator = train_gen,
                    steps_per_epoch = train_samples_steps,
                    epochs=epochs,
                    validation_data = test_gen,
                    validation_steps = val_samples_steps,
                    callbacks = callbacks)
endTime = datetime.now()
print("Code Running Time : ", endTime-startTime)

Generating a batch of given data --->
Epoch 1/20
210/210 [==============================] - ETA: 0s - loss: 1.3284 - acc: 0.1774Generating a batch of given data --->

Epoch 1: val_loss improved from inf to 1.18006, saving model to /content/drive/MyDrive/nlp/Eng_Tel_translation_model_history2.h5
210/210 [==============================] - 57s 227ms/step - loss: 1.3284 - acc: 0.1774 - val_loss: 1.1801 - val_acc: 0.1804 - lr: 0.0010
Epoch 2/20
210/210 [==============================] - ETA: 0s - loss: 1.1427 - acc: 0.2165
Epoch 2: val_loss improved from 1.18006 to 1.11522, saving model to /content/drive/MyDrive/nlp/Eng_Tel_translation_model_history2.h5
210/210 [==============================] - 45s 213ms/step - loss: 1.1427 - acc: 0.2165 - val_loss: 1.1152 - val_acc: 0.2366 - lr: 0.0010
Epoch 3/20
210/210 [==============================] - ETA: 0s - loss: 1.0806 - acc: 0.2544
Epoch 3: val_loss improved from 1.11522 to 1.07027, saving model to /content/drive/MyDrive/nlp/Eng_Tel_translation_

**Model Saving**

### **Testing Model**

In [38]:
Eng_Tel_class.encoder_inputs = encoder_inputs
Eng_Tel_class.encoder_states = encoder_states
Eng_Tel_class.model = model
Eng_Tel_class.inferenceStage()

In [39]:
results = Eng_Tel_class.evaluate(X_test, Y_test)
pass

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 23ms/step


In [40]:
results

{'Acutal_Sentences': ['గత సంవత్సరం టామ్ స్థూల ఆదాయం ముప్పై వేల డాలర్లకు పైగా ఉంది',
  'నేను నిన్ను మళ్ళీ ముద్దు పెట్టుకోవాలనుకుంటున్నాను',
  'టామ్ సహాయం చేయగలడని నేను అనుకున్నాను',
  'మీరు లేకుండా నేను ఒంటరిగా ఉంటాను',
  'ఆమె చదవడానికి ఇష్టపడుతుంది',
  'టామ్ భార్య కవలలతో గర్భవతి',
  'ఆమె గుడ్డిగా జన్మించింది',
  'వాతావరణం కారణంగా నేను ఇంట్లోనే ఉన్నాను',
  'ఇప్పుడు బయటకు వెళ్ళడం అసాధ్యం',
  'మనమందరం కలిసి ఎందుకు పాడము',
  'మనలో చాలా మందికి కోపం వస్తుంది',
  'మీరు మీ ప్రణాళికకు అంటుకుంటున్నారా',
  'ఇది చాలా ఖరీదైనది కాదు',
  'టామ్\u200cను అతని భార్య పక్కన ఖననం చేశారు',
  'నాకు దుప్పటి ఉందా',
  'టామ్ ఆపిల్ తినడం కొనసాగించాడు',
  'టామ్ వంటలు కడుగుతున్నాడు',
  'టామ్ వెనక్కి వణుకుతున్నాడు',
  'టామ్ వయసు పెరుగుతోంది',
  'నేను వేచి ఉండాల్సిందని టామ్ చెప్పాడు',
  'అతను తన తోటలో బంగాళాదుంపలను పెంచాడు',
  'మీ పాలు తాగడం ముగించండి',
  'నాకు కుక్కలు కూడా ఇష్టం',
  'మీకు నా సోదరుడు తెలియదు',
  'ప్రయత్నం లేకుండా ఎవరూ ఏమీ సాధించలేరు',
  'తడి బట్టలు వేలాడదీయడానికి హోటల్ బాత్రూంలో ఒక తీగను అందించింది',


In [25]:
as_words = results["Acutal_Sentences"][0].split()
ps_words = results["Pred_Sentences"][0].split()

In [41]:
results['bleuScore']

0.37020533129945743

In [ ]:
count=0
for w in as_words:
  if w in ps_words:
    count+=1
print(count/len(as_words))

0.6666666666666666
